# Giriş işlemleri

In [1]:
import os
is_colab = False # eğer colab kullanıyorsanız True yapın
if is_colab:
    from google.colab import drive
    drive.mount('/content/drive')
root_path = os.getcwd()  # dosyanın bulunduğu klasör
print(root_path)

c:\Users\kayra\OneDrive\Masaüstü\gsm8k-tr-semantik-analiz\benzerlik_olcumu


## Gerekli kütüphaneleri yükle

In [2]:
import subprocess

# gereksinimler dosyasının yolu
requirements_path = os.path.join(root_path, "gereksinimler.txt")

# gereksinimler yükleniyor
try:
    print("Gereksinimler yükleniyor...")
    subprocess.run(["pip", "install", "-r", requirements_path], check=True)
    print("Gereksinimler başarıyla yüklendi.")
except subprocess.CalledProcessError:
    print("Gereksinimler yüklenirken hata oluştu.")
except FileNotFoundError:
    print(f"Dosya bulunamadı: {requirements_path}")

Gereksinimler yükleniyor...
Gereksinimler başarıyla yüklendi.


## Dosya dizinini sistem yoluna ekle

In [3]:
import sys

sys.path.append(root_path)
print(f"'{root_path}' kütüphane arama yoluna eklendi.")

'c:\Users\kayra\OneDrive\Masaüstü\gsm8k-tr-semantik-analiz\benzerlik_olcumu' kütüphane arama yoluna eklendi.


## Yerel kod dosyalarını içe aktar

In [4]:
from dosya_islemleri import (save_smilarity_json, load_model, load_dataset, save_top1_top5_results_json,
                            get_all_top1_top5_results, read_embedding_from_file)
from benzerlik_islemleri import generate_similarity_json
from basari_hesapla import evaluate_similarity_results_top1_top5
from gorsellestir import visualize_top1_top5_scores, plot_two_tsne_results
from gomme_islemleri import calculate_and_save_raw_embeddings_from_dataset, tsne_sonuc_olustur

C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cihaz tipini al

In [5]:
import torch
# cihaz tipini al
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Kullanılan cihaz tipi: {device}")

Kullanılan cihaz tipi: cpu


## Modelleri tanımla

In [6]:
model_names = [
    "intfloat/multilingual-e5-base",                                # 278M  - 81
    "ibm-granite/granite-embedding-107m-multilingual",              # 107M  - 48
    "intfloat/multilingual-e5-small",                               # 118M  - 36
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",  # 118M  - 62
    "shibing624/text2vec-base-multilingual",                        # 118M  - 77 
    "ytu-ce-cosmos/turkish-colbert"
    ]

In [7]:
# veri kümesi yükleniyor
df = load_dataset()

Veri kümesi yükleniyor: c:\Users\kayra\OneDrive\Masaüstü\gsm8k-tr-semantik-analiz\benzerlik_olcumu\..\gsm8k_tr_1000_soru_cevap.csv


# Gömme (embedding) hesaplama işleri

In [8]:
def get_save_prefix(model_name):
    return model_name.replace("/", "_").replace("-", "_")

In [9]:
# her model için
for i, model_name in enumerate(model_names):
        save_prefix = get_save_prefix(model_name)
        model, tokenizer = load_model(model_name, device_type=device)
        calculate_and_save_raw_embeddings_from_dataset(model, tokenizer, df, save_prefix, device)
        print(f"Embeddingler hesaplandı ve kaydedildi: {model_name} ({i+1}/{len(model_names)})")

Model yükleniyor: intfloat/multilingual-e5-base
Model yüklendi. intfloat/multilingual-e5-base
Hali hazırda hesaplanmış gömme vektörleri sayısı: 10, bu kayıtlar atlanıyor.


Embeddingler hesaplandı ve kaydedildi: intfloat/multilingual-e5-base (1/6)
Model yükleniyor: ibm-granite/granite-embedding-107m-multilingual
Model yüklendi. ibm-granite/granite-embedding-107m-multilingual
Hali hazırda hesaplanmış gömme vektörleri sayısı: 10, bu kayıtlar atlanıyor.


Embeddingler hesaplandı ve kaydedildi: ibm-granite/granite-embedding-107m-multilingual (2/6)
Model yükleniyor: intfloat/multilingual-e5-small
Model yüklendi. intfloat/multilingual-e5-small
Hali hazırda hesaplanmış gömme vektörleri sayısı: 10, bu kayıtlar atlanıyor.


Embeddingler hesaplandı ve kaydedildi: intfloat/multilingual-e5-small (3/6)
Model yükleniyor: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Model yüklendi. sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Hali hazırda hesaplanmış gömme vektörleri say

In [10]:
for i, model_name in enumerate(model_names):
    save_prefix = get_save_prefix(model_name)
    raw_embeddings = read_embedding_from_file(save_prefix)
    tsne_res = tsne_sonuc_olustur(raw_embeddings, save_prefix)

intfloat_multilingual_e5_base_embeddings.json dosyasından 10 gömme vektörü yüklendi.


C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Sistem belirtilen dosyayı bulamıyor
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info 

{'question_tsne': [[30.617822647094727, 38.71656036376953], [-64.2856216430664, 67.18730926513672], [-29.82176399230957, -76.21954345703125], [92.66986083984375, 70.10812377929688], [29.564456939697266, -29.018329620361328], [-92.70587158203125, -20.584510803222656], [57.30401611328125, -91.79039764404297], [100.97412872314453, -11.847574234008789], [-28.611312866210938, 5.789306640625], [6.392165184020996, 104.78252410888672]], 'answer_tsne': [[6.844691753387451, 25.683488845825195], [-37.21583938598633, -12.532829284667969], [-76.36651611328125, 34.67007064819336], [-23.00589370727539, 78.47595977783203], [18.005414962768555, -31.7690486907959], [-87.70413208007812, -44.23259353637695], [51.719993591308594, -80.44207763671875], [54.7404899597168, 61.99224090576172], [75.35906982421875, -5.4599199295043945], [-24.815906524658203, -81.47010803222656]]}
ibm_granite_granite_embedding_107m_multilingual_embeddings.json dosyasından 10 gömme vektörü yüklendi.


C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


{'question_tsne': [[61.528770446777344, -18.536584854125977], [-25.10970687866211, 72.57115173339844], [-20.638076782226562, -76.1319580078125], [16.50522232055664, 25.177413940429688], [-45.68723678588867, 21.023984909057617], [78.82183074951172, 34.167694091796875], [-63.21091842651367, -31.668575286865234], [36.13517761230469, 78.66814422607422], [-0.8418039083480835, -22.68918800354004], [40.54199981689453, -69.98980712890625]], 'answer_tsne': [[63.66225051879883, -32.0129508972168], [6.128839492797852, 27.596576690673828], [-52.4829216003418, 92.97329711914062], [-19.986616134643555, -34.15001678466797], [84.89846801757812, 39.72806930541992], [-54.10647201538086, -100.23040008544922], [28.171241760253906, -97.4785385131836], [27.681489944458008, 98.65149688720703], [-106.44525146484375, -38.425167083740234], [-72.6404037475586, 24.562789916992188]]}
intfloat_multilingual_e5_small_embeddings.json dosyasından 10 gömme vektörü yüklendi.


...

{'question_tsne': [[-55.6584587097168, -23.2010498046875], [-11.248010635375977, -61.88471603393555], [36.781883239746094, -38.87085723876953], [-11.05298900604248, -18.010501861572266], [-60.046409606933594, 27.398141860961914], [-18.21750259399414, 68.77552032470703], [23.681108474731445, 7.211060523986816], [67.12735748291016, 8.295083045959473], [31.477008819580078, 53.122798919677734], [-15.482966423034668, 24.743019104003906]], 'answer_tsne': [[-31.579633712768555, 52.21339416503906], [9.072994232177734, -54.20167541503906], [51.81285095214844, -34.21683120727539], [10.61655330657959, 28.496004104614258], [-33.06374740600586, -32.899635314941406], [54.433929443359375, 8.351869583129883], [-33.595550537109375, 9.686641693115234], [9.924874305725098, -10.470388412475586], [53.17959213256836, 50.993499755859375], [11.054781913757324, 72.27876281738281]]}
shibing624_text2vec_base_multilingual_embeddings.json dosyasından 10 gömme vektörü yüklendi.


C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


{'question_tsne': [[-44.75437545776367, -26.838930130004883], [31.569923400878906, 108.20736694335938], [-120.982666015625, -1.3668043613433838], [105.15827941894531, -68.60214233398438], [3.6970055103302, 34.28583526611328], [-79.87738800048828, -99.40131378173828], [32.18369674682617, -38.14228057861328], [-68.34842681884766, 75.20792388916016], [92.96630859375, 29.42051887512207], [11.009664535522461, -117.89617919921875]], 'answer_tsne': [[15.69243049621582, 43.13095474243164], [-62.58502960205078, -90.50871276855469], [-111.13275146484375, -16.7154483795166], [57.775230407714844, 113.39982604980469], [67.81376647949219, -30.0003662109375], [-73.80517578125, 53.722007751464844], [106.23483276367188, 39.627197265625], [-21.27787208557129, -19.772659301757812], [-30.160974502563477, 120.4164047241211], [25.45491600036621, -97.34648895263672]]}
ytu_ce_cosmos_turkish_colbert_embeddings.json dosyasından 10 gömme vektörü yüklendi.


C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
C:\Users\kayra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


{'question_tsne': [[-27.709613800048828, -15.781874656677246], [-17.347984313964844, -82.14608764648438], [-68.49173736572266, 42.795684814453125], [57.74216079711914, -78.2323989868164], [-87.97549438476562, -35.81516647338867], [-12.409127235412598, 98.60273742675781], [5.194575309753418, 37.41020584106445], [34.79443359375, -17.816913604736328], [65.67726135253906, 63.86739730834961], [97.90076446533203, -2.9899327754974365]], 'answer_tsne': [[32.57017517089844, -9.24068546295166], [40.093746185302734, 27.270353317260742], [-28.099660873413086, 44.46619415283203], [-17.367544174194336, 16.147382736206055], [-46.79160690307617, 9.182398796081543], [7.2076029777526855, -36.729095458984375], [11.889820098876953, 17.8735408782959], [6.473148822784424, 48.399410247802734], [-29.465761184692383, -21.25612449645996], [-1.2015702724456787, -8.250852584838867]]}
